In [90]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from utils.scraper import scrape_recipe,extract_nutrition


In [81]:
url = 'https://preppykitchen.com/wprm_print/cream-pie'
url = 'https://preppykitchen.com/buttermilk-pie'
# url = 'https://preppykitchen.com/cream-pie/'
url = "https://preppykitchen.com/apple-pie/"
try:
    # Send a GET request to the base page
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error fetching {url}: {e}")
    
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')


In [82]:
# Extract image URL
video_url = soup.find('div', class_='rll-youtube-player')['data-src'] if soup.find('div', class_='rll-youtube-player') else None
print(video_url)

https://www.youtube.com/embed/PzFo8G6YNz0


In [87]:
# Extract image URL
image_url = soup.find('div', class_='wprm-recipe-image').find('img')
print(image_url)

<img alt="A white pie dish with an apple pie inside with a slice cut out. An apple and stack of plates in the back." class="attachment-150x150 size-150x150" data-lazy-sizes="(max-width: 150px) 100vw, 150px" data-lazy-src="https://preppykitchen.com/wp-content/uploads/2023/08/Apple-Pie-Recipe-Recipe-Card-330x330.jpg" data-lazy-srcset="https://preppykitchen.com/wp-content/uploads/2023/08/Apple-Pie-Recipe-Recipe-Card-330x330.jpg 330w, https://preppykitchen.com/wp-content/uploads/2023/08/Apple-Pie-Recipe-Recipe-Card-360x361.jpg 360w, https://preppykitchen.com/wp-content/uploads/2023/08/Apple-Pie-Recipe-Recipe-Card-500x500.jpg 500w, https://preppykitchen.com/wp-content/uploads/2023/08/Apple-Pie-Recipe-Recipe-Card-96x96.jpg 96w" data-pin-media="https://preppykitchen.com/wp-content/uploads/2023/08/Apple-Pie-Recipe-Recipe-Card.jpg" decoding="async" height="150" src="data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%20150%20150'%3E%3C/svg%3E" style="border-width: 0p

In [ ]:

"""Find all nutrition containers"""
soup = None
nutrition_elements = soup.select('.wprm-nutrition-label-text-nutrition-container')
print

# Initialize an empty dictionary to store the nutrition data
nutrition_dict = {}

# Loop through each element and extract the nutrition details
for element in nutrition_elements:
    # Extract the label (e.g., Calories, Carbohydrates)
    label = element.select_one('.wprm-nutrition-label-text-nutrition-label').text.strip(': ')
    
    # Extract the value (e.g., 443, 76)
    value = element.select_one('.wprm-nutrition-label-text-nutrition-value').text
    
    # Extract the unit (e.g., kcal, g, mg)
    unit = element.select_one('.wprm-nutrition-label-text-nutrition-unit').text
    
    # Store the data in the dictionary
    nutrition_dict[label] = f"{value} {unit}"
    
    



{'Calories': '443 kcal', 'Carbohydrates': '76 g', 'Protein': '5 g', 'Fat': '15 g', 'Saturated Fat': '6 g', 'Polyunsaturated Fat': '2 g', 'Monounsaturated Fat': '6 g', 'Trans Fat': '0.1 g', 'Cholesterol': '31 mg', 'Sodium': '185 mg', 'Potassium': '247 mg', 'Fiber': '6 g', 'Sugar': '43 g', 'Vitamin A': '215 IU', 'Vitamin C': '9 mg', 'Calcium': '28 mg', 'Iron': '2 mg'}


In [76]:
import re

url = "https://preppykitchen.com/wp-content/uploads/2024/12/Banana-Waffles-Recipe-Card-330x330.jpg"
match = re.search(r'([^/]+\.jpg)$', url)

if match:
    jpg_name = match.group(1)
    print(jpg_name)  # Output: Banana-Waffles-Recipe-Card-330x330.jpg
else:
    print("No match found")


Banana-Waffles-Recipe-Card-330x330.jpg


In [52]:
import json
import requests
from tqdm import tqdm  # Import tqdm for the progress bar

# Open and read the JSON file
with open('dataScraped/sitemapURLS.json', 'r') as file:
    data = json.load(file)


data['post-sitemap']

sitesParsed = 0
for site in data['post-sitemap'][1:10]:
    # match = re.search(r'([^/]+\.jpg)$', url)
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(site, headers=headers)
        response.raise_for_status()
        
        if response.url != 'https://preppykitchen.com':
            sitesParsed += 1

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
    


In [71]:
import json
import requests
from tqdm import tqdm  # Import tqdm for the progress bar

# Open and read the JSON file
with open('dataScraped/sitemapURLS.json', 'r') as file:
    data = json.load(file)

# Get the URLs to process
sites_to_process = data['post-sitemap'][1:10]

sites_parsed = 0

# Use tqdm to show the progress bar
print("Processing sites:")
for site in tqdm(sites_to_process, desc="Progress", unit=" site"):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(site, headers=headers)
        response.raise_for_status()

        if response.url != 'https://preppykitchen.com':
            sites_parsed += 1

    except requests.exceptions.RequestException as e:
        print(f"\nError fetching {site}: {e}")

print(f"\nFinished processing {len(sites_to_process)} sites.")
print(f"Successfully parsed {sites_parsed} sites.")

    


Processing sites:


Progress: 100%|██████████| 9/9 [00:00<00:00,  9.10 site/s]


Finished processing 9 sites.
Successfully parsed 9 sites.
